In [4]:
import google.generativeai as genai

# You can get the API key from Google AI Studio and set it up securely in Colab
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-2.5-flash')

In [5]:
# A small, simulated dataset of support tickets
support_tickets = [
    "My internet connection keeps dropping every few minutes, and I can't stay on a video call. It seems to be a hardware issue with the router.",
    "I need to change my billing address. The last payment was declined. Please help me update my account details.",
    "The software crashes every time I try to save a new document. I'm using version 2.1 on Windows 10.",
    "My account is locked, and I can't reset my password. I'm not receiving the password reset email.",
    "I'd like to request a new feature: a dark mode option for the mobile app.",
    "I have been charged twice for the same subscription, and I want a refund for one of the payments.",
    "My screen is frozen and the application is not responding. I've tried restarting but it didn't help.",
    "I am unable to access my files after the latest software update. The files are gone from the dashboard.",
]

# Define the list of possible tags
possible_tags = [
    "Technical Issue",
    "Billing and Payments",
    "Account Management",
    "Software Bug",
    "Feature Request",
    "Network Issue",
    "Refund Request",
]

In [6]:
def zero_shot_tagging(ticket_text, possible_tags):
    tags_list = ", ".join(possible_tags)
    prompt = f"""
    Analyze the following support ticket and provide the top 3 most probable tags from the list provided.
    The output should be a comma-separated list of tags, without any additional text or formatting.

    Possible tags: {tags_list}

    Support Ticket:
    "{ticket_text}"

    Top 3 tags:
    """
    response = model.generate_content(prompt)
    try:
        # Some simple post-processing to get the list of tags
        # and handle potential malformed output from the model
        tags = [tag.strip() for tag in response.text.split(',')[:3]]
    except (AttributeError, IndexError):
        tags = ["Could not tag"]
    return tags

# Run the zero-shot tagging on a sample ticket
for ticket in support_tickets:
    tags = zero_shot_tagging(ticket, possible_tags)
    print(f"Ticket: '{ticket}'")
    print(f"Zero-Shot Tags: {', '.join(tags)}\n")

Ticket: 'My internet connection keeps dropping every few minutes, and I can't stay on a video call. It seems to be a hardware issue with the router.'
Zero-Shot Tags: Network Issue, Technical Issue, Software Bug

Ticket: 'I need to change my billing address. The last payment was declined. Please help me update my account details.'
Zero-Shot Tags: Billing and Payments, Account Management, Technical Issue

Ticket: 'The software crashes every time I try to save a new document. I'm using version 2.1 on Windows 10.'
Zero-Shot Tags: Software Bug, Technical Issue, Network Issue

Ticket: 'My account is locked, and I can't reset my password. I'm not receiving the password reset email.'
Zero-Shot Tags: Account Management, Technical Issue, Software Bug

Ticket: 'I'd like to request a new feature: a dark mode option for the mobile app.'
Zero-Shot Tags: Feature Request, Technical Issue, Software Bug

Ticket: 'I have been charged twice for the same subscription, and I want a refund for one of the pay

In [7]:
def few_shot_tagging(ticket_text, possible_tags):
    tags_list = ", ".join(possible_tags)
    prompt = f"""
    Analyze the following support tickets and provide the top 3 most probable tags from the list provided.
    The output should be a comma-separated list of tags, without any additional text or formatting.

    Possible tags: {tags_list}

    Examples:
    Support Ticket: "My keyboard is not working after the latest update."
    Top 3 tags: Software Bug, Technical Issue, Hardware Issue

    Support Ticket: "I was charged twice for my monthly subscription. Can you please issue a refund?"
    Top 3 tags: Billing and Payments, Refund Request, Technical Issue

    Support Ticket: "I want to suggest a new feature for the mobile application to export data."
    Top 3 tags: Feature Request, Software Bug, Technical Issue

    Support Ticket:
    "{ticket_text}"

    Top 3 tags:
    """
    response = model.generate_content(prompt)
    try:
        tags = [tag.strip() for tag in response.text.split(',')[:3]]
    except (AttributeError, IndexError):
        tags = ["Could not tag"]
    return tags

# Run the few-shot tagging on the same sample tickets
for ticket in support_tickets:
    tags = few_shot_tagging(ticket, possible_tags)
    print(f"Ticket: '{ticket}'")
    print(f"Few-Shot Tags: {', '.join(tags)}\n")

Ticket: 'My internet connection keeps dropping every few minutes, and I can't stay on a video call. It seems to be a hardware issue with the router.'
Few-Shot Tags: Network Issue, Technical Issue, Software Bug

Ticket: 'I need to change my billing address. The last payment was declined. Please help me update my account details.'
Few-Shot Tags: Billing and Payments, Account Management, Technical Issue

Ticket: 'The software crashes every time I try to save a new document. I'm using version 2.1 on Windows 10.'
Few-Shot Tags: Software Bug, Technical Issue, Network Issue

Ticket: 'My account is locked, and I can't reset my password. I'm not receiving the password reset email.'
Few-Shot Tags: Account Management, Technical Issue, Software Bug

Ticket: 'I'd like to request a new feature: a dark mode option for the mobile app.'
Few-Shot Tags: Feature Request, Software Bug, Technical Issue

Ticket: 'I have been charged twice for the same subscription, and I want a refund for one of the payments